# Prepare our Json files

**Installing scrapy library**

In [ ]:
!pip install scrapy

In [ ]:
%%writefile jobs_1.py


import scrapy

class JobsSpider(scrapy.Spider):
    # Spider name
    name = "jobs"

    # List of starting URLs
    start_urls = [
        'https://tunisia.tanqeeb.com/s/jobs/jobs-in-tunisia'
    ]

    # Custom settings for the spider
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'COOKIES_ENABLED': True,
        'DOWNLOAD_DELAY': 1  # Set the delay to 1 second (or adjust as needed)
    }

    # Initialize the page count
    page_count = 0
    
    # Maximum number of pages to scrape
    max_pages = 50

    # Callback function that processes the response
    def parse(self, response):
        # Check if the response status is 403 (Forbidden)
        if response.status == 403:
            self.logger.warning("Received a 403 Forbidden response. You may be blocked. Check the website's terms of service.")
            return

        # Extract job information from each 'div.card-body' element
        for job in response.css('div.card-body'):
            yield {
                'location': job.css('p.h10 > span:nth-child(1)::text').get(),
                'jobTitle': job.css('h5::text').get(),
                'company': job.css('p.h10 > span:nth-child(2)::text').get(),
                'Add Date': job.css('p.h10 > span:nth-child(3)::text').get(),
            }
        
        # Increment the page count
        self.page_count += 1

        # Check if we have scraped 300 pages, and if not, follow pagination links
        if self.page_count < self.max_pages:
            yield from response.follow_all(css='li.page-item.active > a', callback=self.parse)


In [ ]:
import os

# Check if the file exists before deleting it
if os.path.exists("jobs_1.json"):
    os.remove("jobs_1.json")

# Run your Scrapy spider
# this might take some time and a long output
!scrapy runspider jobs_1.py -o jobs_1.json

In [ ]:
%%writefile jobs_2.py

import scrapy

# Define a Scrapy spider class named 'jobsSpider'
class jobsSpider(scrapy.Spider):
    # Set the name of the spider
    name = "jobs"

    # Define the starting URLs for the spider
    start_urls = [
        "https://www.farojob.net/jobs"
    ]

    # Initialize a page counter
    page_counter = 0

    # Maximum number of pages to scrape
    max_pages = 50

    # Callback function to process the web page's response
    def parse(self, response):
        # Check if the response status is 403 (Forbidden)
        if response.status == 403:
            self.logger.warning("Received a 403 Forbidden response. You may be blocked. Check the website's terms of service.")
            return
            
            
        # Increment the page counter
        self.page_counter += 1

        # Extract job information from each 'article.loadmore-item' element
        for job in response.css('article.loadmore-item'):
            yield {
                'location': job.css('div.loop-item-content > p > span.job-location > a > em::text').get(),
                'jobTitle': job.css(' div.loop-item-content > h2 > a::text').get(),
                'company': job.css('div.loop-item-content > p > span.job-company > a::text').get(),
                'Add Date': job.css('div.loop-item-content > p > span.job-date > time > span::text').get(),
            }
        
        # Follow pagination links to continue scraping other pages, but stop if we've reached the maximum number of pages
        if self.page_counter < self.max_pages:
            yield from response.follow_all(css='div.pagination.list-center > a.next.page-numbers', callback=self.parse)


In [ ]:
import os

# Check if the file exists before deleting it
if os.path.exists("jobs_2.json"):
    os.remove("jobs_2.json")

# Run your Scrapy spider
!scrapy runspider jobs_2.py -o jobs_2.json

# Preparing our DataFrame

In [ ]:
import pandas as pd

# Define a data cleaning function
def clean_data(df):
    if df is not None:
        df['Add Date'] = df['Add Date'].str.replace('\n', '')
        df['company'] = df['company'].str.replace('\n', '')
        df['jobTitle'] = df['jobTitle'].str.replace('\n', '')
        df['location'] = df['location'].str.replace('\n', '')
    return df

# Attempt to read the JSON file
try:
    df1 = pd.read_json("jobs_1.json")
    df1 = clean_data(df1)

except Exception as e:
    print("Error reading jobs_1.json:", e)
    df1 = None

# Attempt to read the JSON file
try:
    df2 = pd.read_json("jobs_2.json")
    df2 = clean_data(df2)
except Exception as e:
    print("Error reading jobs_2.json:", e)
    df2 = None



# Check if dataframes were successfully loaded
if df1 is not None and df2 is not None:
    MyDataFrame = pd.concat([df1, df2], ignore_index=True, sort=False, axis=0)
    display(MyDataFrame)
    print("****** Information about our DataFrame ******* ")
    print(df1.info())
    print(type(MyDataFrame))
else:
    print("Dataframes could not be loaded due to JSON errors.")


# Creating a corpus

In [ ]:
# Initialize an empty list to store job titles
corpus = []

# Iterate through the 'jobTitle' column of MyDataFrame
for i in MyDataFrame['jobTitle']:
    # Check if the value is not None
    if i is not None:
        # Append the non-None job title to the corpus list
        corpus.append(i)

# Print the corpus (list of job titles)
print(corpus)

# Remove Noise Function For our DataFrame

In [ ]:
!pip install nltk

In [ ]:
# Import necessary libraries from nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

# Download required resources (punkt tokenizer and stopwords data)
nltk.download('punkt')
nltk.download('stopwords')

# Create a list of stopwords for both English and French languages
final_stopwords_list = stopwords.words('english') + stopwords.words('french')

# Define a function to remove noise from text
def remove_noise(text, stop_words=final_stopwords_list):
    # Tokenize the input text into words
    tokens = word_tokenize(text)
    cleaned_tokens = []
    for token in tokens:
        # Remove any non-alphanumeric characters
        token = re.sub('[^A-Za-z0-9]+', '', token)
        if len(token) > 1 and token.lower() not in stop_words:
            # Convert the token to lowercase and add it to the cleaned tokens list
            cleaned_tokens.append(token.lower())
    
    # Join the cleaned tokens into a single string
    cleaned_tokens = ' '.join(cleaned_tokens)
    return cleaned_tokens


# Creating tfidf_matrix

In [ ]:
!pip install scikit-learn

In [ ]:
# Import the TfidfVectorizer class from scikit-learn's feature_extraction.text module
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TfidfVectorizer object
# - TfidfVectorizer is a tool for converting a collection of raw documents (in this case, job titles) to a matrix of TF-IDF features.
# - The 'tokenizer' parameter is set to 'remove_noise', which is a custom tokenization function that preprocesses the text data.
tfidf_vectorizer = TfidfVectorizer(tokenizer=remove_noise)

# Use the .fit_transform() method of the TfidfVectorizer
# - The .fit_transform() method processes the 'corpus' data (a list of job titles) and converts it into a TF-IDF matrix.
# - TF-IDF stands for Term Frequency-Inverse Document Frequency, which is a numerical statistic used in natural language processing.
# - It represents the importance of each word in relation to the entire collection of job titles.
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# Print the TF-IDF matrix
# - The TF-IDF matrix contains numerical values that represent the importance of words in each job title.
# - Each row corresponds to a job title, and each column corresponds to a unique word in the entire collection.
# - The values in the matrix are the TF-IDF scores for each word in each job title.
print(tfidf_matrix)


# Clustering with Kmean

In [ ]:
# Import the KMeans clustering algorithm from scikit-learn
from sklearn.cluster import KMeans

# Initialize KMeans with 3 centroids (clusters)
kmeans = KMeans(n_clusters=3, random_state=47)
# 'n_clusters' specifies the number of clusters to create.
# 'random_state' is used to ensure reproducibility of results.

# Fit the KMeans model to the TF-IDF matrix
kmeans.fit(tfidf_matrix)
# This step applies the KMeans clustering algorithm to the TF-IDF matrix created earlier.
# It finds cluster assignments for each data point (job title).

# Store the cluster labels in a variable
clusters = kmeans.labels_
# After fitting the model, 'kmeans.labels_' contains the cluster assignment for each job title.

# Print the cluster labels
print(clusters)

# Create a DataFrame to store job titles and their cluster labels
MyDataFrame = pd.DataFrame(data=corpus, columns=['Job Title'])
# 'corpus' contains the job titles, and we create a DataFrame column named 'Job Title' to store them.

# Add the cluster labels to the DataFrame
MyDataFrame["Labels"] = clusters
# We add a new column 'Labels' to the DataFrame to store the cluster assignments.

# Print information about the DataFrame
print(MyDataFrame.info())
# This line prints basic information about the DataFrame, such as column names and data types.

# Sort the DataFrame based on the 'Labels' column
MyDataFrame = MyDataFrame.sort_values('Labels')
# This step sorts the DataFrame based on the cluster labels.
# It groups job titles by their assigned clusters.

# Print the sorted DataFrame
print(MyDataFrame)

# Print the type of the DataFrame
print(type(MyDataFrame))
# This line prints the type of the DataFrame, which should be 'pandas.core.frame.DataFrame'.


# Trying to visiualise Data

In [ ]:
# Import necessary libraries for data visualization and dimensionality reduction
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

# Initialize PCA (Principal Component Analysis) with 2 components
pca = PCA(n_components=2, random_state=42)
# PCA is used to reduce the dimensionality of the data while preserving important information.
# In this case, it reduces the TF-IDF matrix to 2 dimensions.

# Fit PCA to the TF-IDF matrix and store the reduced vectors in 'pca_vecs'
pca_vecs = pca.fit_transform(tfidf_matrix.toarray())
# The 'toarray()' method converts the sparse TF-IDF matrix to a dense NumPy array.

# Print the reduced PCA vectors
print("********************************\n", pca_vecs)

# Extract the first and second dimensions from the reduced vectors
x0 = pca_vecs[:, 0]
x1 = pca_vecs[:, 1]

# Add cluster assignments and PCA vectors to the DataFrame
MyDataFrame['cluster'] = clusters
MyDataFrame['x0'] = x0
MyDataFrame['x1'] = x1

# Print the DataFrame with cluster assignments and PCA vectors
print(MyDataFrame)

# Define a custom color palette for the clusters
knn_palette = sns.color_palette(['#000C1F', '#29757A', '#FF5050'])
# This defines a color palette with three distinct colors for the clusters.

# Create a scatterplot to visualize the data
plt.figure(figsize=(9, 9))
sns.set()
sns.scatterplot(x='x0', y='x1',
                data=MyDataFrame,
                hue='cluster',
                palette=knn_palette,
                # Define markers for each cluster ('o', '^', 's')
                markers=[',', '^', 'P'],
                # Specify that the style parameter is based on the 'cluster' column
                style='cluster',
                # Increase the size of the points
                s=100,
                # Show the legend
                legend=True
                )
plt.show()
# This code creates a scatterplot to visualize the data in two dimensions after PCA.

# Creating a spark context and reading the data

In [ ]:
!pip install pyspark

In [ ]:
# Import necessary libraries for working with PySpark and Pandas
import pandas as pd
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

# Create or retrieve a SparkContext (sc) instance
sc = SparkContext.getOrCreate()
# SparkContext is the entry point for Spark functionality. It manages the resources for your Spark application.

# Create a SparkSession (spark) using the SparkContext
spark = SparkSession(sc)
# SparkSession provides a single entry point to interact with Spark functionality, including creating DataFrames.

# Read JSON data from 'jobs_1.json' and 'jobs_2.json' into Pandas DataFrames
df1 = pd.read_json("jobs_1.json")
df2 = pd.read_json("jobs_2.json")

# Concatenate the two Pandas DataFrames vertically (along rows) and ignore index
# This combines the data from 'jobs_1.json' and 'jobs_2.json' into a single DataFrame
DataFrame = pd.concat([df1, df2], ignore_index=True, sort=True, axis=0)

# Create a PySpark DataFrame ('jobs_dataFrame') from the 'jobTitle' column of 'DataFrame'
jobs_dataFrame = spark.createDataFrame(DataFrame['jobTitle'].to_frame())
# This creates a PySpark DataFrame with a single column named 'jobTitle' from the 'DataFrame' Pandas DataFrame.

# Print the type of 'jobs_dataFrame'
print(type(jobs_dataFrame))
# This line prints the type of the 'jobs_dataFrame' variable, which should be a PySpark DataFrame.

# Show the first 15 rows of the 'jobs_dataFrame' DataFrame without truncating column data
jobs_dataFrame.show(n=15, truncate=False)
# This displays the first 15 rows of the 'jobs_dataFrame' DataFrame without truncating the column data.

# Creating a pipeline nd Clustering Using Kmean Algo

In [ ]:
# Import necessary libraries and modules from PySpark
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.clustering import KMeans

# Download stopwords data using NLTK
nltk.download('stopwords')

# Create a Tokenizer transformer that converts 'jobTitle' text to lowercase and splits it by white spaces
tokenizer = Tokenizer(inputCol="jobTitle", outputCol="tokens")

# Remove rows with null values from 'jobs_dataFrame'
df = jobs_dataFrame.dropna()

# Create a list of stopwords for both English and French languages
final_stopwords_list = stopwords.words('english') + stopwords.words('french')

# Create a StopWordsRemover transformer to remove stopwords from 'tokens' column
remover = StopWordsRemover(inputCol="tokens", outputCol="stopWordsRemovedTokens", stopWords=final_stopwords_list)

# Calculate Term Frequency (TF) using HashingTF with a specified number of features (numFeatures)
hashingTF = HashingTF(inputCol="tokens", outputCol="rawFeatures", numFeatures=200)

# Calculate Inverse Document Frequency (IDF)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

# Apply the K-Means clustering algorithm with k=3 (3 clusters)
kmeans = KMeans(k=3)

# Create a Pipeline with a sequence of stages for data processing and modeling
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, kmeans])

# Fit the pipeline model to the DataFrame 'df'
model = pipeline.fit(df)

# Transform the DataFrame 'df' using the trained pipeline model
results = model.transform(df)

# Display the results (show the DataFrame)
display(results)

# Print the first 25 rows of the results DataFrame
results.show(25)